In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture   
%pwd


In [3]:
%%capture
cd ..

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import psycopg2 as ps2
import os
import pandas as pd
import numpy as np
import csv
from src.funcs.utils import *

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# pull in db credentials from .env file
db_name=os.getenv('DB_NAME')
db_user=os.getenv('DB_USER')
db_host=os.getenv('DB_HOST_IP')
db_pwd=os.getenv('DB_PWD')

In [ ]:
# conn.close()
# conn.rollback()

In [8]:
# define db session
conn = ps2.connect(
    host = db_host,
    database = db_name,
    user = db_user,
    password = db_pwd)

### Relationships

#### Query for data

##### Directional

In [8]:
# define query
# sql="SELECT DISTINCT \
control.id, inkey[1], controltype, ontology, relationship, effect, mechanism, num_refs, outkey[1] , reference.id, biomarkertype, celllinename, celltype, \
changetype, organ, organism, quantitativetype, tissue  \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null"

In [22]:
# define query
# sql="SELECT DISTINCT \
sql="SELECT \
control.id, inkey[1], controltype,  string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), \
num_refs, outkey[1] , reference.id, string_agg(distinct(biomarkertype), ', ') , string_agg(distinct(celllinename), ', '), \
string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), string_agg(distinct(organ), ', '), \
string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', '),  \
string_agg(distinct(nct_id), ', '),  string_agg(distinct(phase), ', ') \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null \
GROUP BY control.id, inkey[1], controltype, num_refs, outkey[1], reference.id"

In [ ]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/relations_query_csl_refs.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

##### Bi-derectional

In [8]:
# sql="SELECT DISTINCT \
control.id, inkey[1], inoutkey, controltype, ontology, relationship, effect, mechanism, num_refs, outkey[1] , reference.id, biomarkertype, celllinename, celltype, \
changetype, organ, organism, quantitativetype, tissue  \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is null and outkey[1] is null"

In [11]:
sql="SELECT DISTINCT \
control.id, inkey[1], inoutkey, controltype, relationship, string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), num_refs, outkey[1], \
reference.id, string_agg(distinct(biomarkertype), ', '), string_agg(distinct(celllinename), ', '), string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), \
string_agg(distinct(organ), ', '), string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', ') \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is null and outkey[1] is null \
GROUP BY control.id, controltype, reference.id"

In [45]:
df_rel.columns=['id1', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [12]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/bidirectional_relations_query_csl.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

CPU times: user 44.1 s, sys: 2.81 s, total: 47 s
Wall time: 1h 1min 57s


##### Attributes

In [ ]:
# sql="SELECT DISTINCT \
id, inkey[1], attributes, relationship, outkey[1] from resnet.control \
WHERE (control.id = control.attributes)";

In [14]:
sql="SELECT DISTINCT \
id, inkey[1], attributes, relationship, outkey[1] from resnet.control \
WHERE (control.id = control.attributes)";

In [15]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/attributes_relations.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

CPU times: user 46.5 s, sys: 3.78 s, total: 50.3 s
Wall time: 2min 50s


#### Load Data

##### Bi-directional

In [17]:
%%time

df_birect=pd.read_csv('./data/processed/bidirectional_relations_query_csl.txt', sep='|', header=None, encoding='utf-8')

df_birect.info(); df_birect.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 18 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       object
 2   2       object
 3   3       object
 4   4       object
 5   5       object
 6   6       object
 7   7       int64 
 8   8       object
 9   9       int64 
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
 15  15      object
 16  16      object
 17  17      object
dtypes: int64(3), object(15)
memory usage: 648.0+ MB
CPU times: user 13.2 s, sys: 333 ms, total: 13.5 s
Wall time: 13.6 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9223367021478908826,None,"[5747547533469447186, 6031869671835980567]",FunctionalAssociation,None,None,None,1,None,-9223367021478908826,None,None,None,None,liver,None,None,None
1,-9223365481314032826,None,"[5445426109658919917, -2285403010002775161]",FunctionalAssociation,None,None,None,1,None,-9223365481314032826,None,None,None,None,leg,None,None,None
2,-9223357200775258657,None,"[1596328558605610184, -8708891732207432195]",CellExpression,None,None,None,11,None,-9223357200775258657,None,MDA-MB-231,None,None,breast,"Homo sapiens, Oryctolagus cuniculus",None,None
3,-9223356961959519979,None,"[4541556823765750071, 3096071376414863663]",CellExpression,None,None,None,1,None,-9223356961959519979,None,A2058,None,None,None,None,None,None
4,-9223351835527058603,None,"[-2894717825806059007, -5196628532512894922]",FunctionalAssociation,None,None,None,2,None,-9223351835527058603,None,None,None,None,thoracolumbar spine,None,None,None


In [18]:
df_birect.columns=['id1', ':START_ID', 'inOutkey', 'type:TYPE', 'relationship', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', 'biomarkertype', 'celllinename', 'celltype',
      'changetype', 'organ', 'organism', 'quantitativetype', 'tissue']

df_birect.head()

,id1,:START_ID,inOutkey,type:TYPE,relationship,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,-9223367021478908826,None,"[5747547533469447186, 6031869671835980567]",FunctionalAssociation,None,None,None,1,None,-9223367021478908826,None,None,None,None,liver,None,None,None
1,-9223365481314032826,None,"[5445426109658919917, -2285403010002775161]",FunctionalAssociation,None,None,None,1,None,-9223365481314032826,None,None,None,None,leg,None,None,None
2,-9223357200775258657,None,"[1596328558605610184, -8708891732207432195]",CellExpression,None,None,None,11,None,-9223357200775258657,None,MDA-MB-231,None,None,breast,"Homo sapiens, Oryctolagus cuniculus",None,None
3,-9223356961959519979,None,"[4541556823765750071, 3096071376414863663]",CellExpression,None,None,None,1,None,-9223356961959519979,None,A2058,None,None,None,None,None,None
4,-9223351835527058603,None,"[-2894717825806059007, -5196628532512894922]",FunctionalAssociation,None,None,None,2,None,-9223351835527058603,None,None,None,None,thoracolumbar spine,None,None,None


In [19]:
first_ids, second_ids = inOutkeys_to_lists(df_birect)

len(first_ids); len(second_ids)
first_ids[:5]; second_ids[:5]

4718418

4718418

['5747547533469447186',
 '5445426109658919917',
 '1596328558605610184',
 '4541556823765750071',
 '-2894717825806059007']

['6031869671835980567',
 '-2285403010002775161',
 '-8708891732207432195',
 '3096071376414863663',
 '-5196628532512894922']

In [20]:
df_birect_1=df_birect.copy()

df_birect_1.drop(columns=['inOutkey', 'id1', 'id2', 'relationship'], inplace=True)

In [22]:
df_birect_1[':START_ID']=first_ids
df_birect_1[':END_ID']=second_ids
df_birect_1.info(); df_birect_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   :START_ID         object
 1   type:TYPE         object
 2   effect            object
 3   mechanism         object
 4   ref_count:int     int64 
 5   :END_ID           object
 6   biomarkertype     object
 7   celllinename      object
 8   celltype          object
 9   changetype        object
 10  organ             object
 11  organism          object
 12  quantitativetype  object
 13  tissue            object
dtypes: int64(1), object(13)
memory usage: 504.0+ MB


,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,5747547533469447186,FunctionalAssociation,None,None,1,6031869671835980567,None,None,None,None,liver,None,None,None
1,5445426109658919917,FunctionalAssociation,None,None,1,-2285403010002775161,None,None,None,None,leg,None,None,None
2,1596328558605610184,CellExpression,None,None,11,-8708891732207432195,None,MDA-MB-231,None,None,breast,"Homo sapiens, Oryctolagus cuniculus",None,None
3,4541556823765750071,CellExpression,None,None,1,3096071376414863663,None,A2058,None,None,None,None,None,None
4,-2894717825806059007,FunctionalAssociation,None,None,2,-5196628532512894922,None,None,None,None,thoracolumbar spine,None,None,None


In [23]:
df_birect_1[':START_ID']=df_birect_1[':START_ID'].astype('int64')
df_birect_1[':END_ID']=df_birect_1[':END_ID'].astype('int64')
df_birect_1['ref_count:int']=df_birect_1['ref_count:int'].astype('int8')

df_birect_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   :START_ID         int64 
 1   type:TYPE         object
 2   effect            object
 3   mechanism         object
 4   ref_count:int     int8  
 5   :END_ID           int64 
 6   biomarkertype     object
 7   celllinename      object
 8   celltype          object
 9   changetype        object
 10  organ             object
 11  organism          object
 12  quantitativetype  object
 13  tissue            object
dtypes: int64(2), int8(1), object(11)
memory usage: 472.5+ MB


In [24]:
df_birect_1=convert_object_to_category(df_birect_1)
df_birect_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
dtypes: category(11), int64(2), int8(1)
memory usage: 204.4 MB


In [25]:
df_birect_1['biomarkertype'].value_counts()
df_birect_1['mechanism'].value_counts()
df_birect_1['changetype'].value_counts()
df_birect_1['quantitativetype'].value_counts()


None    4718418
Name: biomarkertype, dtype: int64

None       4655608
surface      62810
Name: mechanism, dtype: int64

None    4718418
Name: changetype, dtype: int64

None    4718418
Name: quantitativetype, dtype: int64

In [26]:
df_birect_1.to_pickle('./data/processed/bidirectional_df.pkl')

##### Directional

In [11]:
%%time

# df_direct=pd.read_csv('./data/processed/relations_query_csl.txt', sep='|', header=None, encoding='utf-8')
df_direct=pd.read_csv('./data/processed/relations_query_csl_refs.txt', sep='|', header=None, encoding='utf-8')


df_direct.info(); df_direct.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       int64 
 2   2       object
 3   3       object
 4   4       object
 5   5       int64 
 6   6       int64 
 7   7       int64 
 8   8       object
 9   9       object
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
 15  15      object
 16  16      object
 17  17      object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB
CPU times: user 24.6 s, sys: 1.92 s, total: 26.5 s
Wall time: 26.8 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell, endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [12]:
df_direct.columns=['id1', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [13]:
df_direct.info(); df_direct.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   id1               int64 
 1   :START_ID         int64 
 2   type:TYPE         object
 3   effect            object
 4   mechanism         object
 5   ref_count:int     int64 
 6   :END_ID           int64 
 7   id2               int64 
 8   biomarkertype     object
 9   celllinename      object
 10  celltype          object
 11  changetype        object
 12  organ             object
 13  organism          object
 14  quantitativetype  object
 15  tissue            object
 16  nct_id            object
 17  phase             object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB


,id1,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell, endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [14]:
df_direct=df_direct.drop(columns=['id1', 'id2'])
df_direct.head()

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,7111680438658927661,Regulation,None,None,1,-3121142714757276759,None,None,None,None,None,None,None,None,None,None
2,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,None,TOV-112D,"B-cell, endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-6414782697893422681,Expression,positive,None,1,-296331965523681380,None,None,T-cell,None,None,None,None,None,None,None


In [15]:
df_direct['phase']=df_direct['phase'].fillna('None')

In [16]:
df_obj = df_direct.select_dtypes(['object'])

for col in list(df_obj.columns):
    df_direct[col] = df_direct[col].apply(lambda x: x.strip())
    df_direct[col]=df_direct[col].astype('category')
    
df_direct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int64   
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(3)
memory usage: 583.5 MB


In [18]:
df_direct['ref_count:int']=df_direct['ref_count:int'].astype('int8')

In [19]:
df_direct.isnull().sum()

:START_ID           0
type:TYPE           0
effect              0
mechanism           0
ref_count:int       0
:END_ID             0
biomarkertype       0
celllinename        0
celltype            0
changetype          0
organ               0
organism            0
quantitativetype    0
tissue              0
nct_id              0
phase               0
dtype: int64

In [20]:
df_direct['type:TYPE'].value_counts()

Regulation               6310394
Expression                972328
DirectRegulation          780719
QuantitativeChange        503905
GeneticChange             466177
MolTransport              282980
MolSynthesis              179200
StateChange               168537
Biomarker                 145749
ClinicalTrial             128920
ProtModification           80243
miRNAEffect                67012
ChemicalReaction           60486
PromoterBinding            44478
FunctionalAssociation          1
Name: type:TYPE, dtype: int64

In [21]:
# df_direct.to_pickle('./data/processed/directional_df.pkl')
df_direct.to_pickle('./data/processed/directional_df_refs.pkl')

##### Attributes

In [41]:
%%time

df_att=pd.read_csv('./data/processed/attributes_relations.txt', sep='|', header=None, encoding='utf-8')

df_att.info(); df_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16009643 entries, 0 to 16009642
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       object
 2   2       int64 
 3   3       object
 4   4       object
dtypes: int64(2), object(3)
memory usage: 610.7+ MB
CPU times: user 14.4 s, sys: 1.83 s, total: 16.2 s
Wall time: 18 s


,0,1,2,3,4
0,-9223367021478908826,None,-9223367021478908826,None,None
1,-9223365986387833915,-4129340824583713637,-9223365986387833915,None,-2983484129356197270
2,-9223365556180770043,7111680438658927661,-9223365556180770043,None,-3121142714757276759
3,-9223365481314032826,None,-9223365481314032826,None,None
4,-9223365466130767478,-8447327320654612427,-9223365466130767478,None,-1646880163763186214


In [42]:
df_att.columns=['id1', ':START_ID', 'id2', 'type:TYPE', ':END_ID']
df_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16009643 entries, 0 to 16009642
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   id1        int64 
 1   :START_ID  object
 2   id2        int64 
 3   type:TYPE  object
 4   :END_ID    object
dtypes: int64(2), object(3)
memory usage: 610.7+ MB


In [43]:
df_att['type:TYPE'].value_counts()

None             14917159
is-a              1062691
similar-to          25740
part-of              3325
derivative-of         728
Name: type:TYPE, dtype: int64

In [44]:
df_att=df_att[df_att['type:TYPE']!='None']
df_att.reset_index(drop=True, inplace=True)
df_att. info(); df_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   id1        1092484 non-null  int64 
 1   :START_ID  1092484 non-null  object
 2   id2        1092484 non-null  int64 
 3   type:TYPE  1092484 non-null  object
 4   :END_ID    1092484 non-null  object
dtypes: int64(2), object(3)
memory usage: 41.7+ MB


,id1,:START_ID,id2,type:TYPE,:END_ID
0,-9223342075146411186,-4504017200233399253,-9223342075146411186,is-a,5843242504706058497
1,-9223309118206184327,-7802325777421990647,-9223309118206184327,similar-to,2474720540421519282
2,-9223305918769076311,-7394874569383578266,-9223305918769076311,is-a,-1259866436866537965
3,-9223268026487397641,-4464184570445238538,-9223268026487397641,is-a,-7977145807157790077
4,-9223262990187842537,3044332985631925343,-9223262990187842537,is-a,5081693930784726829


In [45]:
df_att=df_att.drop(columns=['id1', 'id2'])
df_att.info(); df_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   :START_ID  1092484 non-null  object
 1   type:TYPE  1092484 non-null  object
 2   :END_ID    1092484 non-null  object
dtypes: object(3)
memory usage: 25.0+ MB


,:START_ID,type:TYPE,:END_ID
0,-4504017200233399253,is-a,5843242504706058497
1,-7802325777421990647,similar-to,2474720540421519282
2,-7394874569383578266,is-a,-1259866436866537965
3,-4464184570445238538,is-a,-7977145807157790077
4,3044332985631925343,is-a,5081693930784726829


In [46]:
for col in list(df_att.columns):
    df_att[col]=df_att[col].apply(lambda x: x.strip())

In [47]:
df_att[':START_ID']=df_att[':START_ID'].astype('int64')
df_att[':END_ID']=df_att[':END_ID'].astype('int64')
df_att['type:TYPE']=df_att['type:TYPE'].astype('category')

df_att.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype   
---  ------     --------------    -----   
 0   :START_ID  1092484 non-null  int64   
 1   type:TYPE  1092484 non-null  category
 2   :END_ID    1092484 non-null  int64   
dtypes: category(1), int64(2)
memory usage: 17.7 MB


In [48]:
df_att.to_pickle('./data/processed/attributes_df.pkl')

### Concatenate Relationships

In [49]:
df_directional=pd.read_pickle('./data/processed/directional_df.pkl')
df_directional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 468.6 MB


In [50]:
df_bidirect=pd.read_pickle('./data/processed/bidirectional_df.pkl')
df_bidirect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
dtypes: category(11), int64(2), int8(1)
memory usage: 189.0 MB


In [51]:
df_att=pd.read_pickle('./data/processed/attributes_df.pkl')
df_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype   
---  ------     --------------    -----   
 0   :START_ID  1092484 non-null  int64   
 1   type:TYPE  1092484 non-null  category
 2   :END_ID    1092484 non-null  int64   
dtypes: category(1), int64(2)
memory usage: 17.7 MB


In [52]:
%%time

df_concat=pd.concat([df_directional, df_bidirect, df_att])

CPU times: user 3.01 s, sys: 2.24 s, total: 5.25 s
Wall time: 6.56 s


In [53]:
df_concat.info(); df_concat.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16002031 entries, 0 to 1092483
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         object  
 2   effect            category
 3   mechanism         object  
 4   ref_count:int     float64 
 5   :END_ID           int64   
 6   biomarkertype     object  
 7   celllinename      object  
 8   celltype          object  
 9   changetype        object  
 10  organ             object  
 11  organism          object  
 12  quantitativetype  object  
 13  tissue            object  
 14  nct_id            category
 15  phase             category
dtypes: category(3), float64(1), int64(2), object(10)
memory usage: 1.8+ GB


,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,Regulation,None,None,1.0,-2983484129356197270,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,7111680438658927661,Regulation,None,None,1.0,-3121142714757276759,None,None,None,None,None,None,None,None,None,None
2,-8447327320654612427,Regulation,None,None,10.0,-1646880163763186214,None,TOV-112D,"B-cell, endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,3670031740130349683,Regulation,positive,None,1.0,-4650140568030604438,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-6414782697893422681,Expression,positive,None,1.0,-296331965523681380,None,None,T-cell,None,None,None,None,None,None,None


In [54]:
df_concat.isnull().sum()

:START_ID                 0
type:TYPE                 0
effect              1092484
mechanism           1092484
ref_count:int       1092484
:END_ID                   0
biomarkertype       1092484
celllinename        1092484
celltype            1092484
changetype          1092484
organ               1092484
organism            1092484
quantitativetype    1092484
tissue              1092484
nct_id              5810902
phase               5810902
dtype: int64

In [55]:
df_concat['ref_count:int'].fillna(0, inplace=True)

In [56]:
df_concat=df_concat.fillna('None')

In [57]:
df_concat['ref_count:int']=df_concat['ref_count:int'].astype('int8')

In [58]:
df_obj = df_concat.select_dtypes(['object'])

for col in list(df_obj.columns):
    df_concat[col] = df_concat[col].apply(lambda x: x.strip())
    df_concat[col]=df_concat[col].astype('category')
    
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16002031 entries, 0 to 1092483
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 917.5 MB


In [59]:
df_concat.reset_index(drop=True, inplace=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16002031 entries, 0 to 16002030
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 795.4 MB


In [ ]:
# df_concat['celllinename'].value_counts()
# df_concat['celltype'].value_counts()

# df_concat['biomarkertype'].value_counts()
# df_concat['mechanism'].value_counts()
# df_concat['changetype'].value_counts()
# df_concat['quantitativetype'].value_counts()
# df_concat['effect'].value_counts()
# df_concat['tissue'].value_counts()
# df_concat['organism'].value_counts()

In [60]:
df_concat.drop_duplicates(inplace=True)
df_concat.reset_index(drop=True, inplace=True)

df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001720 entries, 0 to 16001719
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 795.4 MB


In [61]:
df_concat['type:TYPE'].value_counts()
df_concat['organ'].value_counts()

Regulation               6310394
FunctionalAssociation    2068008
CellExpression           1492256
Binding                  1157797
is-a                     1062397
Expression                972328
DirectRegulation          780719
QuantitativeChange        503905
GeneticChange             466177
MolTransport              283319
MolSynthesis              179200
StateChange               168537
Biomarker                 145749
ClinicalTrial             128920
ProtModification           80243
miRNAEffect                67012
ChemicalReaction           60489
PromoterBinding            44478
similar-to                 25740
part-of                     3325
derivative-of                727
Name: type:TYPE, dtype: int64

None                                                                                                                     9662169
liver                                                                                                                     263333
brain                                                                                                                     246320
heart                                                                                                                     156469
skin                                                                                                                      128663
                                                                                                                          ...   
airway, alveolus, colon, esophagus, gastrointestinal tract, intestine, lung, mammary gland, nerve, nose, pleura, skin          1
blood vessel, neural tube, placenta                                                              

In [62]:
df_concat=df_concat.replace('None', '_')
df_concat.info(); df_concat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001720 entries, 0 to 16001719
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 795.4 MB


,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,Regulation,_,_,1,-2983484129356197270,_,_,_,_,intestine,Rattus norvegicus,_,_,_,_
1,7111680438658927661,Regulation,_,_,1,-3121142714757276759,_,_,_,_,_,_,_,_,_,_
2,-8447327320654612427,Regulation,_,_,10,-1646880163763186214,_,TOV-112D,"B-cell, endothelial cell",_,peripheral nervous system,_,_,_,_,_
3,3670031740130349683,Regulation,positive,_,1,-4650140568030604438,_,_,plasma cell,_,_,_,_,plasma,_,_
4,-6414782697893422681,Expression,positive,_,1,-296331965523681380,_,_,T-cell,_,_,_,_,_,_,_


In [68]:
# df cleanup

df_concat['type:TYPE']=df_concat['type:TYPE'].apply(lambda x: x.upper())
df_concat.info(); df_concat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001720 entries, 0 to 16001719
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 795.4 MB


,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,REGULATION,_,_,1,-2983484129356197270,_,_,_,_,intestine,Rattus norvegicus,_,_,_,_
1,7111680438658927661,REGULATION,_,_,1,-3121142714757276759,_,_,_,_,_,_,_,_,_,_
2,-8447327320654612427,REGULATION,_,_,10,-1646880163763186214,_,TOV-112D,"B-cell, endothelial cell",_,peripheral nervous system,_,_,_,_,_
3,3670031740130349683,REGULATION,positive,_,1,-4650140568030604438,_,_,plasma cell,_,_,_,_,plasma,_,_
4,-6414782697893422681,EXPRESSION,positive,_,1,-296331965523681380,_,_,T-cell,_,_,_,_,_,_,_


In [69]:
df_concat['type:TYPE'].value_counts()

REGULATION               6310394
FUNCTIONALASSOCIATION    2068008
CELLEXPRESSION           1492256
BINDING                  1157797
IS-A                     1062397
EXPRESSION                972328
DIRECTREGULATION          780719
QUANTITATIVECHANGE        503905
GENETICCHANGE             466177
MOLTRANSPORT              283319
MOLSYNTHESIS              179200
STATECHANGE               168537
BIOMARKER                 145749
CLINICALTRIAL             128920
PROTMODIFICATION           80243
MIRNAEFFECT                67012
CHEMICALREACTION           60489
PROMOTERBINDING            44478
SIMILAR-TO                 25740
PART-OF                     3325
DERIVATIVE-OF                727
Name: type:TYPE, dtype: int64

In [ ]:
# filter ref count =0 if type not Clinical trial

# import numpy as np
# idx =np.where((df_rel_deduped['ref_count:int']==0) & (df_rel_deduped['type:TYPE']!='CLINICALTRIAL') & \
#               (df_rel_deduped['type:TYPE']!='IS-A') & (df_rel_deduped['type:TYPE']!='PART-OF') & \
#              (df_rel_deduped['type:TYPE']!='DERIVATIVE-OF') & (df_rel_deduped['type:TYPE']!='SIMILAR-TO'))  #get index values to filter out
# df_rel_filtered=df_rel_deduped[~df_rel_deduped.index.isin(idx[0])]
# df_rel_filtered.info(); df_rel_filtered.head()

In [70]:
cols=list(df_directional.columns)
cols

[':START_ID',
 'type:TYPE',
 'effect',
 'mechanism',
 'ref_count:int',
 ':END_ID',
 'biomarkertype',
 'celllinename',
 'celltype',
 'changetype',
 'organ',
 'organism',
 'quantitativetype',
 'tissue',
 'nct_id',
 'phase']

In [71]:
df_concat=df_concat[cols]
df_concat.head(5)

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,REGULATION,_,_,1,-2983484129356197270,_,_,_,_,intestine,Rattus norvegicus,_,_,_,_
1,7111680438658927661,REGULATION,_,_,1,-3121142714757276759,_,_,_,_,_,_,_,_,_,_
2,-8447327320654612427,REGULATION,_,_,10,-1646880163763186214,_,TOV-112D,"B-cell, endothelial cell",_,peripheral nervous system,_,_,_,_,_
3,3670031740130349683,REGULATION,positive,_,1,-4650140568030604438,_,_,plasma cell,_,_,_,_,plasma,_,_
4,-6414782697893422681,EXPRESSION,positive,_,1,-296331965523681380,_,_,T-cell,_,_,_,_,_,_,_


In [72]:
%%time
df_concat.to_csv('./data/processed/relations.txt', sep='|', index=False, header=True)

CPU times: user 3min 5s, sys: 996 ms, total: 3min 6s
Wall time: 3min 8s


In [73]:
# list of column names for header file
cols=list(df_concat.columns)
cols

[':START_ID',
 'type:TYPE',
 'effect',
 'mechanism',
 'ref_count:int',
 ':END_ID',
 'biomarkertype',
 'celllinename',
 'celltype',
 'changetype',
 'organ',
 'organism',
 'quantitativetype',
 'tissue',
 'nct_id',
 'phase']

In [74]:
# create headers file
df_rel_headers=pd.DataFrame(columns=cols)
df_rel_headers
df_rel_headers.to_csv('./data/processed/relations_header.txt', sep= '|', index=False)

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase


### Nodes

In [85]:
# define query
sql='select id, name, nodetype from resnet.node where id is not null and name is not null and nodetype is not null'

In [86]:
%%time

# save query results to file
with conn.cursor() as cur:
    with open('./data/raw/nodes_.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter='|')
        for record in cur.fetchall():
            line="|".join(map(str, record))+'\n'
            
            f.write(line)

CPU times: user 3.06 s, sys: 383 ms, total: 3.44 s
Wall time: 3.96 s


#### Create Node files

In [94]:
df_node=pd.read_csv('./data/raw/nodes_.txt', delimiter='|', header=None, encoding='utf-8')
df_node.info(); df_node.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470805 entries, 0 to 1470804
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1470805 non-null  int64 
 1   1       1470805 non-null  object
 2   2       1470805 non-null  object
dtypes: int64(1), object(2)
memory usage: 33.7+ MB


,0,1,2
0,1000020775393047472,r_LOC108352567,Protein
1,-1000022879358845591,Rx26449263,SmallMol
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SemanticConcept
3,1000026386881640747,rs876660037,GeneticVariant
4,-1000028657865034959,m_Fbxw19,Protein


In [95]:
# create column headers
df_node.columns=[':ID', 'name', ':LABEL']
df_node.head()

,:ID,name,:LABEL
0,1000020775393047472,r_LOC108352567,Protein
1,-1000022879358845591,Rx26449263,SmallMol
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SemanticConcept
3,1000026386881640747,rs876660037,GeneticVariant
4,-1000028657865034959,m_Fbxw19,Protein


In [96]:
# df cleanup
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.upper())
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.strip())
df_node['name']=df_node['name'].apply(lambda x: x.strip())
df_node[':ID']=df_node[':ID'].apply(lambda x: str(x).strip())  #convert to string and strip()
df_node[':ID']=df_node[':ID'].astype('int64')   # convert back to int
df_node.info(); df_node.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470805 entries, 0 to 1470804
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   :ID     1470805 non-null  int64 
 1   name    1470805 non-null  object
 2   :LABEL  1470805 non-null  object
dtypes: int64(1), object(2)
memory usage: 33.7+ MB


,:ID,name,:LABEL
0,1000020775393047472,r_LOC108352567,PROTEIN
1,-1000022879358845591,Rx26449263,SMALLMOL
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SEMANTICCONCEPT
3,1000026386881640747,rs876660037,GENETICVARIANT
4,-1000028657865034959,m_Fbxw19,PROTEIN


In [97]:
# check for name strings that are too long
df_node['name_len']=df_node['name'].apply(lambda x: len(x))

In [98]:
df_node.sort_values(by='name_len', ascending=False)

,:ID,name,:LABEL,name_len
1107832,-7235442027224814239,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...,SMALLMOL,279728
600508,-4382490639255431495,"D-Glucose, O-4,6-dideoxy-4-((4-(alpha-D-glucop...",SMALLMOL,254
7778,1043483757283375332,"2,4a-Dihydroxy-1-methyl-8-methylenegibb-3-ene-...",SMALLMOL,248
1229720,7946627188054156171,"Cholan-24-oic acid, 3,7,12-trioxo-, (5beta)-, ...",SMALLMOL,243
377137,-3119824026281474218,"1,3,5-Triazine-2,4,6-triamine, N,N'''-1,2-etha...",SMALLMOL,216
...,...,...,...,...
205050,2153331758759806501,HN,CELLTYPE,2
1086727,-7115660023470151832,c,SMALLMOL,1
1273144,-8190334285627176943,X,SMALLMOL,1
432989,-3437747827402286845,\,SMALLMOL,1


In [104]:
# Issue is leading quote with no ending quote in sql db for id -7235442027224814239 above
# the following is a temp fix unitil resolved in sql db
# process long name field

x=df_node[df_node[':ID']==-7235442027224814239]['name']
long_index=df_node[df_node[':ID']==-7235442027224814239].index[0] #capture index to delete later
long_index
x=x.to_frame().reset_index()
x.info(); x.head()
y=x.loc[0,'name']
len(y)

1107832

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   1 non-null      int64 
 1   name    1 non-null      object
dtypes: int64(1), object(1)
memory usage: 144.0+ bytes


,index,name
0,1107832,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...


279728

In [106]:
# parse bad field
# lst=y.split('\r\n')
lst=y.split('\n')

lst[:5]
lst[0]='-7235442027224814239|' + lst[0]
lst[:5]

['1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-1-yl)butyl)-4-(4-fluorophenyl)piperidine-2,6-dione|SmallMol',
 '7235446264322303181|Rx25148080|SmallMol',
 '7235446778373378569|6-chloro-2-fluoropurine|SmallMol',
 '-7235449901734945744|2,2,7-trimethyl-4H-1,4-benzoxazin-3-one|SmallMol',
 '-7235456322816637391|N-(p-Tolyl)-p-toluenesulphonamide|SmallMol']

['-7235442027224814239|1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-1-yl)butyl)-4-(4-fluorophenyl)piperidine-2,6-dione|SmallMol',
 '7235446264322303181|Rx25148080|SmallMol',
 '7235446778373378569|6-chloro-2-fluoropurine|SmallMol',
 '-7235449901734945744|2,2,7-trimethyl-4H-1,4-benzoxazin-3-one|SmallMol',
 '-7235456322816637391|N-(p-Tolyl)-p-toluenesulphonamide|SmallMol']

In [107]:
# create list of lists from long name field
new_list=[]
for i in range(len(lst)):
    z=lst[i].split('|')
    new_list.append(z)
    

In [108]:
len(new_list)
new_list[:5]

3855

[['-7235442027224814239',
  '1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-1-yl)butyl)-4-(4-fluorophenyl)piperidine-2,6-dione',
  'SmallMol'],
 ['7235446264322303181', 'Rx25148080', 'SmallMol'],
 ['7235446778373378569', '6-chloro-2-fluoropurine', 'SmallMol'],
 ['-7235449901734945744',
  '2,2,7-trimethyl-4H-1,4-benzoxazin-3-one',
  'SmallMol'],
 ['-7235456322816637391', 'N-(p-Tolyl)-p-toluenesulphonamide', 'SmallMol']]

In [109]:
# create df from list of lists
df_temp=pd.DataFrame(new_list, columns=[':ID', 'name', ':LABEL'])
df_temp.info(); df_temp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3855 entries, 0 to 3854
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   :ID     3855 non-null   object
 1   name    3855 non-null   object
 2   :LABEL  3854 non-null   object
dtypes: object(3)
memory usage: 90.5+ KB


,:ID,name,:LABEL
0,-7235442027224814239,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...,SmallMol
1,7235446264322303181,Rx25148080,SmallMol
2,7235446778373378569,6-chloro-2-fluoropurine,SmallMol
3,-7235449901734945744,"2,2,7-trimethyl-4H-1,4-benzoxazin-3-one",SmallMol
4,-7235456322816637391,N-(p-Tolyl)-p-toluenesulphonamide,SmallMol


In [110]:
df_temp.dropna(inplace=True)
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3854 entries, 0 to 3853
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   :ID     3854 non-null   object
 1   name    3854 non-null   object
 2   :LABEL  3854 non-null   object
dtypes: object(3)
memory usage: 120.4+ KB


In [111]:
df_temp[':LABEL']=df_temp[':LABEL'].apply(lambda x: x.upper())
df_temp.head()

,:ID,name,:LABEL
0,-7235442027224814239,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...,SMALLMOL
1,7235446264322303181,Rx25148080,SMALLMOL
2,7235446778373378569,6-chloro-2-fluoropurine,SMALLMOL
3,-7235449901734945744,"2,2,7-trimethyl-4H-1,4-benzoxazin-3-one",SMALLMOL
4,-7235456322816637391,N-(p-Tolyl)-p-toluenesulphonamide,SMALLMOL


In [112]:
# combine dfs
df_node.drop(int(f'{long_index}'), inplace = True) #drop row with long name field
df_node.drop(columns=['name_len'], inplace=True)
df_node_new=pd.concat([df_node, df_temp])
df_node_new.reset_index(inplace=True, drop=True)
df_node_new.info(); df_node_new.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474658 entries, 0 to 1474657
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   :ID     1474658 non-null  object
 1   name    1474658 non-null  object
 2   :LABEL  1474658 non-null  object
dtypes: object(3)
memory usage: 33.8+ MB


,:ID,name,:LABEL
0,1000020775393047472,r_LOC108352567,PROTEIN
1,-1000022879358845591,Rx26449263,SMALLMOL
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SEMANTICCONCEPT
3,1000026386881640747,rs876660037,GENETICVARIANT
4,-1000028657865034959,m_Fbxw19,PROTEIN


In [113]:
# check for presence of ';;' & ';'
df_node_new[df_node_new['name'].str.contains(";")]
df_node_new[df_node_new['name'].str.contains(";;")]

,:ID,name,:LABEL
94,-1000445907920769387,Ac-FRPX(Hle; D)F(3-Cl)F-NH2,SMALLMOL
103,1000511471434219628,(S)-3-chloro-4-(N-(3-fluoropropyl)pyrrolidin-3...,SMALLMOL
1038,10055389442503362,Carbamoyl-C(D;bridge C11)KFFW(D)F(4-(Isopropyl...,SMALLMOL
1301,-1006961062563094732,Ac-ED(lactam K7)HX(2-Nal; D)RWK-NH2,SMALLMOL
1503,1008036727165805856,X(Iqc; D)C(bridge X7)YW(D)RTX(Pen)T-NH2,SMALLMOL
...,...,...,...
1473596,7251653840688617755,X(Inp)F(D;4-(Chloro))X(2-Nal;D)F-Amino,SMALLMOL
1473919,-7253500055656412392,m_T(3;10)61H (Gene ID 21159),PROTEIN
1474113,7254604792324956166,YF(D)GW(D)X(Nle; N-Me)DF-NH2,SMALLMOL
1474252,725545692487216447,"2(S)2-{[6-(5-chloro-1H-pyrrolo[2,3-b]pyridin-3...",SMALLMOL


,:ID,name,:LABEL
8757,-1048773164875225821,"Y(2,6-Me; N-CO)X(Orn(D)modifixfffd;; lactam G5...",SMALLMOL
946734,6327831653333845569,X(Orn(D)modifixfffd;; lactam G4)X(2-Nal)P(D)G,SMALLMOL


In [114]:
# convert single and double semi-colons to colons
df_node_new['name']=df_node_new['name'].str.replace(';;', ':')
df_node_new['name']=df_node_new['name'].str.replace(';', ':')


In [115]:
df_node_new.to_csv('./data/processed/nodes.txt', sep='|', index=False, header=True)

In [116]:
# create list of column names for header file
cols=list(df_node.columns)
cols

[':ID', 'name', ':LABEL']

In [117]:
# create and save header file
node_headers=pd.DataFrame(columns=cols)
node_headers
node_headers.to_csv('./data/processed/nodes_header.txt', sep='|', index=False)

,:ID,name,:LABEL


In [50]:
conn.close()

**To load data into Neo4j**, click on 'Add' in the project pane. Name the new project, click on the '...' by the 'Open' button for the project, and choose 'Terminal'. Take note of the dbms number at the cursor. Copy the nodes.txt, nodes_header.txt, relations.txt, and realations_header.txt files pepared above. Drop them in the import file found here: C:\Users\\[user_name]\\.Neo4jDesktop\relate-data\dbmss\dbms-##(number at cursor)\import

At the Neo4j terminal, change to the bin directory (type 'cd bin') and then paste the following at the cursor:
neo4j-admin import --delimiter="|" --nodes=import/nodes_header.txt, import/nodes.txt --relationships=import/relations_header.txt, import/relations.txt --skip-bad-relationships=true

Go back to the project pane, press 'Start'. The active database will now be shown in a pane at the top. Press open to use in the Neo4j browser.